<a href="https://colab.research.google.com/github/arnoldjames98/arnoldjames98.github.io/blob/main/systemKinematics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Biodegradable Seed Spreading Robot](https://arnoldjames98.github.io/)
# System Kinematics

### 1. Create a figure (either in python, in a vector-based drawing program like inkscape or illustrator, or as a solidworks rendering) of your system kinematics.

TODO: Describe figure and add correct measurements to table

![Mechanism Drawing](https://raw.githubusercontent.com/arnoldjames98/arnoldjames98.github.io/main/images/assignment3/mechanism.png)

| Length | Rigid Body | Measurement (cm)
| --- | --- | --- |
| L10 | Yellow | 2 |
| L02 | Purple | 2 |
| L42 | Purple | 2 |
| L40 | Purple | 2 |
| L31 | Red | 2 |
| L61 | Red | 2 |
| L63 | Red | 2 |
| L54 | Green | 2 |
| L32 | Orange | 2 |
| L56 | Blue | 2 |
| L5E | Blue | 2 |
| LE6 | Blue | 2 |


### 2. Make the device in paper or cardboard. You need an up-to-date model if it has changed from your individual assignments. The paper model should dimensionally match your code.

TODO: Upload image of device to github and add images here. May need to add a brief description too.

### 3. Using a pynamics-based script, develop a kinematic model for your device.

In [1]:
%matplotlib inline
!pip install pynamics

import all required packages

In [2]:
import pynamics
from pynamics.frame import Frame
from pynamics.variable_types import Differentiable,Constant
from pynamics.system import System
#from pynamics.body import Body
#from pynamics.dyadic import Dyadic
from pynamics.output import Output,PointsOutput
#from pynamics.particle import Particle
import pynamics.integration
import sympy
import numpy
import matplotlib.pyplot as plt
plt.ion()
from math import pi
#from pynamics.constraint import Constraint
import scipy.optimize

Create a pynamics system

In [3]:
system = System()
pynamics.set_system(__name__,system)

Declare all length constants

In [4]:
# TODO: Add correct measurements when defining length constants
# We may also considering our angle constraints for the triangular links here

# From Loop Closure Eq. #1
l_10 = Constant(2,'l_10',system)
l_31 = Constant(1.5,'l_31',system)
l_32 = Constant(1,'l_32',system)
l_02 = Constant(1,'l_02',system)

# From Loop Closure Eq. #2
#l_10 already is included in Eq. 1
l_61 = Constant(2,'l_61',system)
l_56 = Constant(2,'l_56',system)
l_54 = Constant(2,'l_54',system)
l_40 = Constant(2,'l_54',system)

# From Loop Closure Eq. #3
#l_32 already is included in Eq. 1
#l_54 already is included in Eq. 2
#l_56 already is included in Eq. 2
l_63 = Constant(2,'l_63',system)
l_42 = Constant(2,'l_42',system)

# End effector
l_E6 = Constant(2,'l_E6',system)
l_E5 = Constant(2,'l_E5',system)

Create the differentiable state variables, cooresponding to the joint angles (we might not need all of these depending on how we do the constraints)

In [5]:
q0,q0_d,q0_dd = Differentiable('q0',system)
q1,q1_d,q1_dd = Differentiable('q1',system)
q2,q2_d,q2_dd = Differentiable('q2',system)
q3,q3_d,q3_dd = Differentiable('q3',system)
q4,q4_d,q4_dd = Differentiable('q4',system)
q5,q5_d,q5_dd = Differentiable('q5',system)
q6,q6_d,q6_dd = Differentiable('q6',system)
qE,qE_d,qE_dd = Differentiable('qE',system)

Initial guess for starting positions

In [15]:
# TODO: We can measure the angles on our figure to make some guesses
initialvalues = {}
initialvalues[q0]=-90*pi/180
initialvalues[q0_d]=0
initialvalues[q1]=90*pi/180
initialvalues[q1_d]=0
initialvalues[q2]=90*pi/180
initialvalues[q2_d]=0
initialvalues[q3]=90*pi/180
initialvalues[q3_d]=0
initialvalues[q4]=90*pi/180
initialvalues[q4_d]=0
initialvalues[q5]=90*pi/180
initialvalues[q5_d]=0
initialvalues[q6]=90*pi/180
initialvalues[q6_d]=0
initialvalues[qE]=90*pi/180
initialvalues[qE_d]=0

Retrieve state variables in the order they are stored in the system

In [7]:
# Not sure if we need this, but it's included in the tutorial
statevariables = system.get_state_variables()

Create the reference frames, with one cooresponding to each rigid link
![Reference Frames](https://raw.githubusercontent.com/arnoldjames98/arnoldjames98.github.io/main/images/assignment3/refFrames.png)

In [8]:
N = Frame('N') # Purple link at p0 (ground)
A = Frame('A') # Yellow link at p1
B = Frame('B') # Green link at p5
C = Frame('C') # Red link at p6
D = Frame('D') # Orange link at p3
E = Frame('E') # Blue link at pE (end-effector)

Declare N as the Newtonian (fixed) frame

In [9]:
system.set_newtonian(N)

Rotate frames about their z-axes so they align with the correct rigid link.

In [10]:
A.rotate_fixed_axis_directed(N,[0,0,1],q1,system) # Yellow
B.rotate_fixed_axis_directed(N,[0,0,1],q5,system) # Green
D.rotate_fixed_axis_directed(N,[0,0,1],q3,system) # Orange

# Define frame C w.r.t to frame A
C.rotate_fixed_axis_directed(A,[0,0,1],q6,system) # Red

# Define frame E w.r.t to frame C
E.rotate_fixed_axis_directed(C,[0,0,1],qE,system) # End-effector

Define vectors for each of the joint locations (not necessarily the vectors connecting two vectors)

In [13]:
# Vectors representing the joint locations w.r.t N frame
# Fixed points on ground link
p0 = 0*N.x
p2 = -6.6*N.x - 1.97*N.y # Coordinates given
p4 = -2.6*N.x + 9.9*N.y # Coordinates given

# Points directly connected to ground
p1 = p0 + l_10*A.x # Yellow
p3 = p2 + l_32*D.x # Orange
p5 = p4 + l_32*B.x # Green

# Additional points
p6 = p1 + l_61*C.x # Red from yellow
pE = p6 + l_E6*E.x # End-effector from red
#print(pE.components)

# List of points to that will be used for plotting
points = [p0, p1, p2, p3, p4, p5, p6, pE]

Define vectors that connect the joint locations

In [12]:
# When subtracting: # Vector to head minus vector to tail

# From Loop Closure Eq. #1
r_10 = p1
r_31 = p3 - p1
r_32 = p3 - p2 
r_02 = -p2

# From Loop Closure Eq. #2
#r_10 already is included in Eq. 1
r_61 = p6 - p1
r_56 = p5 - p6
r_54 = p5 - p4
r_40 = p4

# From Loop Closure Eq. #3
#r_32 already is included in Eq. 1
#r_54 already is included in Eq. 2
#r_56 already is included in Eq. 2
r_63 = p6 - p3
r_42 = p4 - p2

# End effector
r_E6 = pE - p6
r_E5 = pE - p5

Create a list of initial values ini0 in the order of the system’s state variables

In [16]:
statevariables = system.get_state_variables()
ini0 = [initialvalues[item] for item in statevariables]

Create all the vector equation constraints (e.g., loop closure equations, rigid link constraints, etc.)

![Loop Closure Equations](https://raw.githubusercontent.com/arnoldjames98/arnoldjames98.github.io/main/images/assignment3/loopClosureEqs.png)

In [18]:
# Loop closure equations (constraint to ensure all loops are equal to zero)
# Equation 1 (R02 + R10 + R31 = R32)
loopClosure1 = r_02 + r_10 + r_31 - r_32

# Equation 2 (R10 + R61 + R56 = R40 + R54)
loopClosure2 = r_10 + r_61 + r_56 - r_40 - r_54

# Equation 3 (R42 + R54 = R32 + R63 + R56)
loopClosure3 = r_42 + r_54 - r_32 - r_63 - r_56

# TODO: Rigid body constraints (how would I we do this?)

Convert the vector constraints into scalar equations that can be solved

In [24]:
eq = []

# Loop closure equations
eq.append((loopClosure1).dot(N.x))
eq.append((loopClosure1).dot(N.y))
eq.append((loopClosure2).dot(N.x))
eq.append((loopClosure2).dot(N.y))
eq.append((loopClosure3).dot(N.x))
eq.append((loopClosure3).dot(N.y))

print(eq)

# Todo: constraint equations
# HERE

eq_d=[(system.derivative(item)) for item in eq]
print(eq_d)

[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
